# AI-MAILs
## 深層学習入門: 手書き数字の分類

Ver. 20220715

## 本セクションの目標
- MNISTデータセットを用いて手書き数字の分類を行うことで、どのように深層学習を実装するかを学ぶ

## 目次
- A. MNISTデータセット
- B. Tensorflow/Keras を用いた深層学習の実装
- C. 学習の可視化 
- D. ハイパーパラメータ

## A. MNISTデータセット
- NIST (National Institute of Standards and Technology) が保有していたデータセットを再構成したデータベース
- 60,000枚の訓練用画像と10,000枚の評価用画像が含まれている

| <img src="https://www.nemotos.net/nb/img/MnistExamples.png" width="300"> |
| --: |
| Wikipediaより引用 |

## B. 深層学習の実装
- 深層学習を実装する手順は以下となる

| <img src="https://www.nemotos.net/nb/img/dl_flow.png" width="300"> |
| --: |
| 動かしながら学ぶPyTorchプログラミング入門より引用 |

- この流れに従っていく

### 1. 必要なパッケージのインポート
- 今回必要なパッケージは以下
    - numpy
    - matplotlib
    - tensorflow
        - keras は tensorflow 2.0 から tensorflow の中に取り込まれた

In [ ]:
# 必要なパッケージのインポート

# NumPy
import numpy as np

# Matplotlib
import matplotlib.pyplot as plt

# Tensorflow
import tensorflow as tf

# ラベルを one-hotベクトルに変換する関数 to_categorical()
from tensorflow.keras.utils import to_categorical

### 2. データの前処理
- MNISTの画像データはひとつひとつのピクセルの値が0-255の値をとる
- これを0-1の値をとるように変換する
- 脳画像の前処理に関しては、午前中の下地先生のセクションがここに該当

#### 2.1. データの読み込みと確認

In [ ]:
# tensorflow の中に mnist データセットが既に入っている
mnist = tf.keras.datasets.mnist

In [ ]:
# mnist.load_data() で訓練データとテストデータにわけて格納する
# mnistは、訓練データとテストデータがそれぞれタプルにわかれて入っている
# 訓練データの画像を train_images, 正解ラベルを train_labels に格納する
# テストデータも同じ
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

In [ ]:
# train_images について確認する
# まず、型から確認
# numpy.ndarray型
type(train_images)

In [ ]:
# shape
# 60000枚の画像、1枚の画像は 28 x 28 で構成
train_images.shape

In [ ]:
# plt.imshow() を使って実際の画像を確認
# for を使って、最初の3人分のデータを見る
for i in range(3):
    plt.figure(figsize=(1,1))
    plt.imshow(train_images[i], cmap='gray')
    plt.show()
    

In [ ]:
# train_labels の内容を確認
# スライシングで train_labels の最初の3つのラベルを取り出す
# 画像とラベルが一致していることを確認
train_labels[0:3]

In [ ]:
# train_labels の shape を確認
# 60000 のデータがある1次元のデータ
train_labels.shape

In [ ]:
# 同様にテストデータも確認
# shape
# 10,000枚の画像、1枚の画像は 28 x 28 で構成
test_images.shape

In [ ]:
# plt.imshow() を使って実際の画像を確認
# for を使って、最初の3人分のデータを見る
for i in range(3):
    plt.figure(figsize=(1,1))
    plt.imshow(test_images[i], cmap='gray')
    plt.show()
    

In [ ]:
# test_labels の内容を確認
# スライシングで test_labels の最初の3つのラベルを取り出す
test_labels[0:3]

#### 2.2 正解ラベルの one-hotベクトル化
- 第3部(1)でみたように、正解ラベルを one-hotベクトルに変換する
- `to_categorical()` 関数を使うことで変換できる

In [ ]:
# train_labels を one-hotベクトルに変換
train_labels = to_categorical(train_labels)

# test_labels も同様に one-hotベクトルに変換
test_labels = to_categorical(test_labels)

In [ ]:
# 新しい train_labels の shape を確認
# 60000行10列の行列になっている
train_labels.shape

In [ ]:
# train_labels の 最初の3つを見てみる
# 正解 5, 0, 4 に相当するところが 1 になっていることに着目
train_labels[0:3]

In [ ]:
# test_labels の 最初の3つも確認する
# 正解 7, 2, 1 に相当するところが 1 になっていることに着目
test_labels[0:3]

#### 2.3 データの正規化
- 深層学習に限らず、データ解析においてデータの範囲をある決まった範囲に変換することを正規化という
- 正規化を行うことで、異なる変数がモデルに与える影響を均等にできる
    - 例: 年齢 (20-80) と 身長 (140-200)
- 今、画像は 0-255 の整数をとるので、これを 0-1 になるように変換する

In [ ]:
# dtype 属性で numpy配列内の数字のデータ型がわかる
# uint8 は unsigned integer 8bit 符号なし 8bit 整数 (0-255)
train_images.dtype

In [ ]:
# train_images の最小値
# min() メソッドを使えばよい
train_images.min()

In [ ]:
# train_images の最大値
# max() メソッドを使えばよい
train_images.max()

In [ ]:
# 0-255で構成されるので、255で割れば、値は 0-1 の間となる
# 255.0 と小数点をつけて割ることで、Pythonは出力を必ず float型としてくれる
# 計算結果を同じ変数名にいれることで、変数を増やすことなく、
train_images = train_images / 255.0
test_images = test_images / 255.0

In [ ]:
# train_images の値が本当に0-1になったか確認
# min() と max() を使えばよい
# 最小値
train_images.min()

In [ ]:
# 最大値
train_images.max()

In [ ]:
# dtypeも確認する
# 今回は float64 倍精度浮動小数点数
train_images.dtype

### 3. 訓練データとテストデータの作成
- MNISTデータセットは手書き数字6万枚の訓練データセットと手書き数字1万枚のテストデータセットから構成されている
- 「**訓練データ**」「**検証データ**」「**テストデータ**」の3つを準備する
    - 訓練データ: ニューラルネットワークのパラメータを決めるためのデータ
    - 検証データ: 訓練データで得られたパラメータがどの程度の精度があるかを検証するためのデータ
    - テストデータ: ニューラルネットワークの汎用性を評価するためのデータ
        - 訓練で使ったものと別のセットを使わないといけない
- Tensorflow には、`model.fit()` メソッドに、`validation_split` という引数が準備されており、ここで訓練データの何割を検証データとして使用するかを設定できる
    - 今回は訓練データの2割を検証データとして使用することとする (`validation_split=0.2`)

### 4. ニューラルネットワークの定義

- ここで、自分がイメージするニューラルネットワークモデルを定義する
- 下図の赤線の部分, すなわち **順伝播 forwad propagation** のモデルを構築

<img src="https://www.nemotos.net/nb/img/dl_overview_4.png" width="400">

- 今は以下のように定義する
    - 層と層の結合は全結合とする
    - 第1層は画像が 28 x 28 で構成されているので、そのピクセル数(784)がユニット数
    - 第2層のユニット数は 128 とする
    - 第2層の活性化関数は **ReLU**関数 とする
    - 過学習を防ぐため、全結合層の2割は drop とする (Dropout=0.2, 明日説明)
    - このモデルとしては、最終の出力は 0-9 の10のクラスを分類したい
    - そのため、第3層は 出力層に渡す準備として、ユニット数は 10 とする
    - 第3層の出力はそのまま出力層の入力とする (Tensorflow ではそのように構築することが勧められている)
    - 出力層の活性化関数は多クラス分類に適した **Softmax**関数 とする

- 活性化関数の特徴

| 関数名 | 特徴 | 
| :-- | :-- |
| ReLU | 隠れ層に使うことで、非線形問題を解くことができるようになる <br> Sigmoid関数は0-1の値しかとらないので層が厚くなるほど誤差が小さくなっていき、<br>入力層まで誤差が伝搬する前に誤差が消失するという勾配消失問題が発生する |
| Sigmoid | 0-1の間の確率で表現可能なため2クラス分類の出力層に用いる |
| Softmax | 各クラスの確率の総和が1となるように正規化された関数であるため多クラス分類の出力層に用いる |

<img src="https://www.nemotos.net/nb/img/nn_model.png" width="400"> 

- これらはTensorflow/Kerasでは以下のように定義できる
    - プリセットで準備されている **tf.keras.Sequential**モデル を選択する(Sequential: 連続する)
    - 入力画像は **Flatten** を使うことでベクトルにできる
    - 全結合層は **Dense** で規定できる

In [ ]:
# モデルを定義
# tf.keras.modls.Sequential([第1層,第2層,第3層,出力層]) と順に記載していく
# リストなので、項目と項目の区切りに , を忘れない(忘れるとエラーになる)

model = tf.keras.models.Sequential([
    # 入力画像の dimension を指定
    tf.keras.layers.Flatten(input_shape=(28, 28)), 
    # 第2層のユニット数を 128 にし、 活性化関数は ReLU とする
    tf.keras.layers.Dense(128, activation='relu'),
    # 過学習防止のため、結合層の20%を dropout
    tf.keras.layers.Dropout(0.2),
    # 第3層のユニット数を 10 にする
    tf.keras.layers.Dense(10),
    # 出力層でSoftmax関数 で処理して結果を出力する
    tf.keras.layers.Softmax()
])

In [ ]:
# 訓練データを使って予測値を計算 (forward propagation)
# model() の後に .numpy() をつけることで、NumPy配列に変換する
predictions = model(train_images).numpy()

In [ ]:
# predictions を表示
# この時点では、モデルを決めただけで、重みはランダムに割り当てられている
# そのため、各クラスの確率はおおよそ 1/10 あたりになるはず
# ひとつの数字の画像が1行、列が 0 - 9 の数字である確率
# 一切学習はしていないことに注意！train_labelsはまだ使われていない
predictions

### 5. 損失関数と最適化関数の定義
- 次に損失関数と最適化関数(オプティマイザ)を決定する
- その後、model.compile() で損失関数と最適化関数をモデルに組み込む
- 下図の赤線の部分、すなわち **逆伝播 back propagation** のモデルを構築

<img src="https://www.nemotos.net/nb/img/dl_overview_5.png" width="400">

- よく用いられる損失関数

| 目的 | 関数名 | Function Name | 損失関数名<br>(Tensorflow) | 損失関数名(PyTorch) |
| :-- | :-- | :-- | :-- | :-- |
| 回帰 | 平均二乗誤差 | Mean Squared Error | mean_squared_error | nn.MSELoss |
| 2クラス分類 | バイナリ交差エントロピー | Binary Cross Entropy | binary_crossentropy | nn.BCELoss |
| 多クラス分類 | ソフトマックス交差エントロピー | Softmax Cross Entropy | categorical_crossentropy (one-hot vector用)<br> sparse_categorical_crossentropy | nn.CrossEntropyLoss |

In [ ]:
# 損失関数には、ソフトマックス交差エントロピー誤差を使用
# 今回は正解ラベルは one-hotベクトル として準備していることから、
# tf.keras.losses.CategoricalCrossentropy() を使用する
# one-hotベクトルでない場合は、
# tf.keras.losses.SparseCategoricalCrossentropy() を使用する
loss_fn = tf.keras.losses.CategoricalCrossentropy()

In [ ]:
# 今の場合、予測値はいずれも 0.1 程度
# 損失は、-log(0.1) ≒ 2.3 程度になるはず
loss_fn(train_labels, predictions).numpy()

In [ ]:
# 参考
# -log(0.1) を計算
-np.log(0.1)

In [ ]:
# 最適化関数として、Adam を選択する
# 損失関数は先程定義した交差エントロピー誤差を使用する
# モデルの評価は accuracy で行う
model.compile(optimizer='adam',
             loss = loss_fn,
             metrics = ['accuracy'])

In [ ]:
# どのようなモデルになったかを model.summary() で知ることができる
model.summary()
# パラメータ数
# Flatten: 入力層なのでなし
# Dense 100480
# 入力層 784 * 第2層 128 + 第2層のそれぞれのユニットに対する定数項 128
# Dense 1290
# 第2層 128 * 第3層 10 + 第3層のそれぞれのユニットに対する定数項 10
# 合計 101,770 ものパラメータをこれから学習させることになる
# パラメータが多いため、パラメータ推定のために必要なデータ数が膨大となる

### 6. 学習・評価
- `model.fit()` で学習させる
- この時、訓練データと訓練データの正解ラベルをモデルに与える
- `validation_split` で訓練データのうち検証に使う割合を指定する
- `batch_size` でバッチサイズを指定する
- `epochs` で何回学習するかを指定する
- 学習の結果を変数 history に代入してあとで可視化する

In [ ]:
# loss: 損失
# accuracy: 正答率
# 10回の繰り返しで、loss が少しずつ減少、accuracy は増加
#
# 訓練データ,訓練データの正解ラベルをまず入力 (train_images, train_labels)
# 訓練データの2割を検証データとして使用 (validation_split=0.2)
# ミニバッチ学習として、バッチサイズは128に設定 (batch_size=128)
# 繰り返し回数は10回 (epochs=10)                            
history = model.fit(train_images,train_labels,
                    validation_split=0.2,
                    batch_size=128,
                    epochs=10)

- `model.evaluate(テストデータ,テストラベル)`を使うことで、modelの性能を表示できる

In [ ]:
# model.evaluateの引数に test_images, test_labels を指定
# verbose =1 とすると、学習のときと同じような結果表示になる
model.evaluate(test_images,test_labels, verbose=1)

## C. 学習の視覚化
- matplotlib を用いて学習の様子を視覚化する
- 変数 history.history の中に loss と accuracy の10回の値が格納されている

In [ ]:
# history.historyの中を見てみる
# ディクショナリ型
# キーが 'loss', 'accuracy', 'val_loss', 'val_accuracy'
# val_ は 検証データでの結果
# 値が 損失値と正答率の推移
history.history

In [ ]:
# 訓練データの損失値 loss と検証データの損失値 val_loss をグラフとして表示
# 訓練データの loss の値を取り出して train_loss に代入
# ディクショナリ型の値は 変数名['キー名']　で取り出せる
train_loss = history.history['loss']
# 検証データの loss を取り出して val_loss に代入
val_loss = history.history['val_loss']

# train_loss と val_loss をプロットする
plt.plot(train_loss, label='training')
plt.plot(val_loss, label='validation')
# グラフのタイトル
plt.title('loss over epochs')
# x軸の名前
plt.xlabel('epochs')
# y軸の名前
plt.ylabel('loss')
# 凡例
plt.legend()
# これらをすべてまとめて表示
plt.show()

In [ ]:
# 訓練データの正答率 accuracy と検証データの正答率 val_accuracy をグラフとして表示
# 訓練データの accuracy を取り出して train_accuracy に代入
train_accuracy = history.history['accuracy']
# 検証データの accuracy を取り出して val_accuracy に代入
val_accuracy = history.history['val_accuracy']

# train_accuracy と val_accuracy をプロットする
plt.plot(train_accuracy, label='training')
plt.plot(val_accuracy, label='validation')
# グラフのタイトル
plt.title('accuracy over epochs')
# x軸の名前
plt.xlabel('epochs')
# y軸の名前
plt.ylabel('accuracy')
# 凡例
plt.legend()
# これらをすべてまとめて表示
plt.show()

## D. ハイパーパラメータ
- 深層学習の実装の例を示したが、自身のデータで解析する際、人間が設定しなければならないパラメータがいくつかある
- これらをハイパーパラメータという
- 具体的には以下のようなものが挙げられる
    - 中間層のユニット数
    - Dropout率
    - 中間層の活性化関数
    - 損失関数
    - 最適化関数
    - バッチサイズ
    - エポック数
- より精度の高いモデルを構築するために、これらを吟味していくことが必要となる

## 練習問題

- 以下のパラメータでモデルを構築し、学習させた時、テストデータの正答率がどう変わるかを見てみてください

- 中間層のユニット数: 64
- バッチサイズ: 16
- エポック数: 5

In [ ]:
# 必要なパッケージのインポート
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

# データの準備
mnist = tf.keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# データの正規化
train_images = train_images / 255.0
test_images = test_images / 255.0

# 変数 model を初期化
model = []

# モデルの構築
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)), 
    tf.keras.layers.Dense(ここに代入, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10),
    tf.keras.layers.Softmax()
])

# 損失関数には、交差エントロピー誤差を使用
# 正解ラベルを one-hot ベクトルに変換していないため、
# SparseCategoricalCrossentropy()を使う
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()

# 最適化関数には、adam を使用
model.compile(optimizer='adam',
             loss = loss_fn,
             metrics = ['accuracy'])

# モデルの要約
model.summary()

In [ ]:
# モデルの学習
history = model.fit(train_images,train_labels,
                    validation_split=0.2,
                    batch_size=ここに代入,
                    epochs=ここに代入)


In [ ]:
# loss の推移
train_loss = history.history['loss']
val_loss = history.history['val_loss']
plt.plot(train_loss, label='training')
plt.plot(val_loss, label='validation')
plt.title('loss over epochs')
plt.xlabel('epochs')
plt.ylabel('loss')
plt.legend()
plt.show()

In [ ]:
# accuracy の推移
train_accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']
plt.plot(train_accuracy, label='training')
plt.plot(val_accuracy, label='validation')
plt.title('accuracy over epochs')
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.legend()
plt.show()

In [ ]:
# テストデータでの評価
model.evaluate(test_images,test_labels, verbose=1)